# Core Read

This Notebook focuses on the process of selecting data.

Column transformations are found in "10 - Column Transformations"

The following topics are covered:
- Joins
- Subquery
- Group By
- Having
- Streaming
- With Hint


Further Reading:
- [SELECT syntax](https://www.sqlite.org/lang_select.html) by SQLite
- ["Using SELECT Statements"](https://docs.sqlalchemy.org/en/20/tutorial/data_select.html) by SQLAlchemy
- ["SQL Statements and Expressions API"](https://docs.sqlalchemy.org/en/20/core/expression_api.html) by SQLAlchemy

## Tables
- Product
- Customer
- Order
- Orderline

In [ ]:
import sqlalchemy as sa
from utils import *

base = sa.MetaData()
Products = sa.Table('products', base, 
                        sa.Column('id', sa.INTEGER, primary_key=True, autoincrement=True),
                        sa.Column('name', sa.VARCHAR(255), nullable=False, index=True),
                        sa.Column('price', sa.DOUBLE, nullable=True)
                    )

Customers = sa.Table('customers', base, 
                        sa.Column('id', sa.INTEGER, primary_key=True, autoincrement=True),
                        sa.Column('name', sa.VARCHAR(255)),
                    )

Orders = sa.Table('orders', base, 
                        sa.Column('id', sa.INTEGER, primary_key=True, autoincrement=True),
                        sa.Column('customer_id', sa.INTEGER, sa.ForeignKey(Products.c['id']), nullable=False)
                 )

OrderLines = sa.Table('orderlines', base, 
                        sa.Column('order_id', sa.INTEGER, sa.ForeignKey(Orders.c['id'],  ondelete='CASCADE'), nullable=False),
                        sa.Column('product_id', sa.INTEGER, sa.ForeignKey(Products.c['id']), nullable=False),
                        sa.Column('quantity', sa.DOUBLE, nullable=False),
                      # order_id and product_id should be unique (as a pair).
                        sa.PrimaryKeyConstraint('order_id', 'product_id'),
                     )

In [ ]:
engine = sa.create_engine('sqlite:///')
con = engine.connect()
base.create_all(engine)

In [ ]:
print(repr(Customers.c.name))
print(repr(Customers.c['name']))
print(repr(Customers.columns.name))
print(repr(Customers.columns['name']))

## Data
Add a little bit of starting data.

In [ ]:
with con.begin():
    con.execute(Customers.insert(), [{'name': 'Alice'}, {'name': 'Bob'}])
    con.execute(Products.insert(), [{'name': 'Cookie', 'price': 1}, { 'name': 'Ice Cream', 'price': 2}])

In [ ]:
with con.begin() as t:
    result = con.execute(Orders.insert(), {'customer_id': 1})
    order_id = result.inserted_primary_key[0]
    result = con.execute(OrderLines.insert(), [{'order_id': order_id, 'product_id': 1, 'quantity': 1}])

# Basic Select

In [ ]:
# Select All columns
print('\n--- A ---')
print(Customers.select())

print('\n--- B ---')
print(sa.select(Customers))

# Select 1 column
print('\n--- C ---')
print( sa.select( Customers.c['id']))

# Where

In [ ]:
query = (
    sa.select(Customers)
    .where(
        # default .where has an implied 'AND'
        Customers.c['id'] == 0, 
        Customers.c['name'] == 'DoesNotExist'
    )
)
print(query)
print('--- SQL start ---')
with logs(), con.begin():
    for row in con.execute(query):
        print('row:', row)
print('--- SQL end ---')

## Logical operators

A lot of expression can be combined.

Operation | Functional | Operator
---|---|---
OR | or_(a, b) | a \| b 
AND | and_(a,b) | a & b
NOT | not_(a) | ~a
IS NULL | a.is_(None) | -
IS NOT NULL | a.is_not(None) | -
Contains | a.in_(tuple_or_expr) | -
Any | any_(a, b, c) | -


In [ ]:
# OR
or_1 = (Customers.c['id'] == 0) | (Customers.c['name'] == 'DoesNotExist')
or_2 = sa.or_(Customers.c['id'] == 0,  Customers.c['name'] == 'DoesNotExist')
print('1)', str(or_1))
print('-----')
print('2)', str(or_2))
print('-----')
query = (
    sa.select(Customers)
    .where(criteria)
)
print(query)
print('--- SQL start ---')
with logs(), con.begin():
    for row in con.execute(query):
        print('row:', row)
print('--- SQL end ---')

# Joins

Many SQL Dialects provide their own shorthands for certain operations.<br>
Remember that when debugging queries.

Additionally, the `RIGHT JOIN` does not exist in SQLAlchemy.<br>
Most SQLAlchemy developers will just tell you to reverse the position of the operands so a `LEFT JOIN` can be used instead.

Developers can build their 'select' statments without defining the join statement beforehand.<br>
This allows for statments to be written a bit more similar to regular SQL.

## Inner Join
The Inner Join is the overlap between two tables.<br>
SQLAlchemy writes this as `(SELECT).join(table, expr)`.

- **Table:** The tableto join with
- **Expr:** The 'on' expression, usually a column comparison.

Developers can build their 'select' statements with defining the join statement beforehand.<br>

```
query = sa.select(Customers.c['name'], Orders.c['order_id'])

```

**Remember:** `JOIN` and `INNER JOIN` are the same thing.


In [ ]:
query = sa.select(
    Customers.c['name'], 
    Orders.c['id'].label('order_id')
)
query = query.join(Customers, Customers.c['id'] == Orders.c['id'])
print(str(query))

In [ ]:
query = (
    Orders.select()
    .join(Customers, Customers.c['id'] == Orders.c['customer_id'])
)
with logs(), con.begin():
    for row in con.execute(q):
        print(row)

## Left Join

The Left Outer Join effectively extends the data of a table with that of another.<br>
The syntax is similar to a regular join: `join(table, expr, isouter=True)`

**Remember:** `LEFT JOIN` and `LEFT OUTER JOIN` are the same thing.

## Outer Join
`FULL OUTER JOIN` and `OUTER JOIN` are the same thing.

`join(table, expr, full=True, isouter=True)`

## Cross Join / Cartesian Product
Allegedly ``join(table, sa.literal(True))`` or ``(Tbl1, Tbl2).all()``

# Subquery

# Group By

In [ ]:
query = sa.select(
        sa.func.count(Customers.c['id']).label('my_count')
    ).group_by(Customers.c['name'])

print(query)

# Having

In [ ]:
# Window

In [ ]:
# UNION (ALL)

In [ ]:
# INTERSECT ?#
# EXCEPT ? https://www.sqlite.org/lang_select.html
# https://docs.sqlalchemy.org/en/20/core/selectable.html

# WITH (expr) -> Common Table Expression (cte)

# Streaming

In [ ]:
# yield_per

# With Hint


In [ ]:
from sqlalchemy.dialects import mssql, sqlite

query = sa.select(Products)
query = query.with_hint(Products, text='WITH(NOLOCK)', dialect_name='mssql')
print('Microsoft SQL Server:')
print(str(query.compile(dialect=mssql.dialect())).replace('\n', ''))
print('SQLite:')
print(str(query.compile(dialect=sqlite.dialect())).replace('\n', ''))
